# **Reto: Modelo de Tráfico con Sistema Multi-Agentes**

## **Descripción:**

La movilidad urbana, se define como la habilidad de transportarse de un lugar a otro1 y es fundamental para el desarrollo económico y social y la calidad de vida de los habitantes de una ciudad. Desde hace un tiempo, asociar la movilidad con el uso del automóvil ha sido un signo distintivo de progreso. Sin embargo, esta asociación ya no es posible hoy. El crecimiento y uso indiscriminado del automóvil —que fomenta políticas públicas erróneamente asociadas con la movilidad sostenible—genera efectos negativos enormes en los niveles económico, ambiental y social en México.

Durante las últimas décadas, ha existido una tendencia alarmante de un incremento en el uso de automóviles en México. Los Kilómetros-Auto Recorridos (VKT por sus siglas en Inglés) se han triplicado, de 106 millones en 1990, a 339 millones en 2010. Ésto se correlaciona simultáneamente con un incremento en los impactos negativos asociados a los autos, como el smog, accidentes, enfermedades y congestión vehicular.

Para que México pueda estar entre las economías más grandes del mundo, es necesario mejorar la movilidad en sus ciudades, lo que es crítico para las actividades económicas y la calidad de vida de millones de personas.

Este reto te permitirá contribuir a la solución del problema de movilidad urbana en México, mediante un enfoque que reduzca la congestión vehicular al simular de manera gráfica el tráfico, representando la salida de un sistema multi agentes.

### **Etapa 1.1:** *Modelación del modelo*

Para modelar los agentes, se utilizó como principal librería a ***mesa***, que da acceso a dos clases fundamentales ***Agent*** y ***Model***, los cuales fungen como planos para definir la funcionalidad tanto del entorno como de cada elemento dentro del mismo. Se observan también clases como ***RandomActivation*** (*que viene del módulo time*), para activar a los agentes y al modelo mismo en cada paso de tiempo, ***MultiGrid*** para actualizar las posiciones de cada uno de los agentes. Este tipo de cuadrículas permite la presencia de más de un tipo de agente, a diferencia de ***Grid***, proveniente del mismo módulo ***space***.

Por otro lado, se importaron módulos propios, de los cuales se encuentran: ***traffic_agents, agent_types, directions*** y ***grid_manager***.
    
- **traffic_agents:** *Describe el comportamiento de los agentes involucrados en el sistema de tráfico.*
- **agent_types:** *Funge como un enumerador para distinguir los tipos de agentes.*
- **directions:** *Funge como un enumerador para distinguir las direcciones de las calles.*
- **grid_manager:** *Se encarga de crear y manejar las instancias de los nodos dentro de un grafo que funciona a manera de cuadrícula.*

In [18]:
from mesa import Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from traffic_agents import *
from agent_types import AgentTypes as agt
from directions import Directions as dirs
from grid_manager import *

Las constantes que definen al modelo son las siguientes:

In [19]:
COLS = 26
ROWS = 26
CARS = 5
FILENAME = "JupyterResources\\map.txt"

***COLS*** es la cantidad de columnas que tendrá la cuadrícula, así como ***ROWS*** representa a la cantidad de filas. Por otra parte, ***CARS*** es  el número de agentes de tipo ***CarAgent***, del que se hablará más adelante. Finalmente, ***FILENAME*** define la ruta de un archivo que describe simbólicamente la región en la que los carros actuarán. El comportamiento definido en el modelo es el siguiente: 

In [ ]:
txt_content = """>>>>>>>>>>>>>>>s>>>>>>>>vv
>>>>>>>>>>>>>>>s>>>>>>>>vv
^^##D###vv######SS#####Dvv
^^######vv######^^######vv
^^######vvD#####^^######vv
^^######vv######^^######vv
^^######vv######^^######vv
^^###D##SS######^^####D#vv
^^s<<<<<<<s<<<<<<<s<<<<<vv
^^s<<<<<<<s<<<<<<<s<<<<<vv
SS######vv######SS######vv
^^######vv######^^######vv
^^######vv######^^######vv
^^D#####vv#####D^^######vv
^^######vv######^^######vv
^^######SS######^^######SS
^^>>>>>s>>>>>>>>>>>>>>>svv
^^>>>>>s>>>>>>>>>>>>>>>svv
^^######vv####D#########vv
^^######vv##############vv
^^#####Dvv#############Dvv
^^######vv##############vv
^^######vv##############vv
^^######SS#########D####vv
^^<<<<<<<<s<<<<<<<<<<<<<<<
^^<<<<<<<<s<<<<<<<<<<<<<<<"""

%mkdir JupyterResources
%cd JupyterResources
%store txt_content >map.txt
%cd ..

class TrafficModel(Model):
    def __init__(self, max_steps: int) -> None:
        # Define model attributes
        self.cols = COLS
        self.rows = ROWS
        self.grid = MultiGrid(COLS, ROWS, False)
        self.schedule = RandomActivation(self)
        self.running = True
        # Define class attributes
        self.cars = CARS
        self.max_steps = max_steps
        self.arrivals = 0
        self.agent_uid = 0
        # Initialize standard car map and destination list
        self.standard_map = make_grid(ROWS, COLS, self)
        self.destinations = []
        self.new_car_spawns = [[(0, 0), (0, 1)], [(self.cols - 1, self.rows - 1), (self.cols - 1, self.rows - 2)], [(0, self.rows - 1), (0, self.rows - 2)], [(self.cols - 1, 0), (self.cols - 1, 1)]]
        # Create agents using a text file
        with open(FILENAME) as m:
            # Initialize agents from txt file
            lines = m.readlines()
            for row in range(len(lines)):
                # For every character in a line, check which agent to make depending on the character
                for col in range(len(lines[row])):
                    agent = None
                    if lines[row][col] == "s":
                        # Create light agent
                        if lines[row][col - 1] == "<" or lines[row][col + 1] == "<":
                            agent = Light(self.agent_uid, self, dirs.LEFT)
                            self.standard_map[row][col].direction = dirs.LEFT
                        elif lines[row][col - 1] == ">" or lines[row][col + 1] == ">":
                            agent = Light(self.agent_uid, self, dirs.RIGHT)
                            self.standard_map[row][col].direction = dirs.RIGHT
                    elif lines[row][col] == "S":
                        # Create light agent
                        if lines[row - 1][col] == "^" or lines[row + 1][col] == "^":
                            agent = Light(self.agent_uid, self, dirs.UP)
                            self.standard_map[row][col].direction = dirs.UP
                        elif lines[row - 1][col] == "v" or lines[row + 1][col] == "v":
                            agent = Light(self.agent_uid, self, dirs.DOWN)
                            self.standard_map[row][col].direction = dirs.DOWN
                    elif lines[row][col] == "#":
                        # Create building agents
                        agent = Building(self.agent_uid, self)
                        self.standard_map[row][col].state = NodeTypes.OBSTACLE
                    elif lines[row][col] == "D":
                        # Create destination point
                        agent = Destination(self.agent_uid, self)
                        self.standard_map[row][col].state = NodeTypes.OBSTACLE
                        self.destinations.append(agent)
                    # If cell is part of the road, assign it's direction
                    elif lines[row][col] == "^":
                        self.standard_map[row][col].direction = dirs.UP
                    elif lines[row][col] == "v":
                        self.standard_map[row][col].direction = dirs.DOWN
                    elif lines[row][col] == "<":
                        self.standard_map[row][col].direction = dirs.LEFT
                    elif lines[row][col] == ">":
                        self.standard_map[row][col].direction = dirs.RIGHT
                    # If agent is not None, add agent to the model's grid
                    if agent:
                        if agent.type_id != agt.BUILDING:
                            self.schedule.add(agent)
                        self.grid.place_agent(agent, (row, col))
                        self.agent_uid += 1
        # Set directions of every node
        init_neighborhood(self.standard_map)

        # Add cars in random positions without crashing them beforehand
        for _ in range(self.cars):
            car_pos = self.get_unique_pos()
            car = Car(self.agent_uid, self, car_pos)
            self.schedule.add(car)
            self.grid.place_agent(car, car_pos)
            self.agent_uid += 1
    
    # Get a position where a cell is empty
    def get_unique_pos(self) -> tuple:
        random_pos = lambda r, c: (self.random.randrange(r), self.random.randrange(c))
        new_pos = random_pos(self.rows, self.cols)
        while not self.grid.is_cell_empty(new_pos):
            new_pos = random_pos(self.rows, self.cols)
        return new_pos

    # Get a destination from the destination list
    def get_unique_destination(self) -> tuple or None:
        if len(self.destinations) > 0:
            return self.destinations.pop()
        else:
            return None

    def step(self) -> None:
        # If the number of cars that have arrived to their destinations are still less than the number of cars, keep running
        if self.schedule.steps < self.max_steps:
            # Advance one step
            self.schedule.step()
            # Change the lights' state every 10 steps
            if self.schedule.steps % 10 == 0:
                for agent in self.schedule.agents:
                    if agent.type_id == agt.LIGHT:
                        agent.state = not agent.state
                # Create car agents in each entrance
                for i in range(len(self.new_car_spawns)):
                    car_pos = self.random.choice(self.new_car_spawns[i])
                    car = Car(self.agent_uid, self, car_pos)
                    self.schedule.add(car)
                    self.grid.place_agent(car, car_pos)
                    self.agent_uid += 1
        else:
            self.running = False

Los atributos del modelo son los siguientes:

- **self.cols**: *Es el número de columnas en ***MultiGrid***.*
- **self.rows**: *Es el número de filas en ***MultiGrid***.*
- **self.grid**: *Es la instancia de ***MultiGrid***.*
- **self.schedule**: *Es la instancia de ***RandomActivation***.*
- **self.running**: *Determina si el programa continúa corriendo.*
- **self.cars**: *Es el número inicial de carros en el modelo.*
- **self.max_steps**: *Es el número máximo de pasos a dar antes de detener el programa.*
- **self.agent_uid**: *Es el identificador único para cada agente.*
- **self.standard_map**: *Es el grafo para que cada carro encuentre el camino más corto a su destino.*
- **self.destinations**: *Es una lista con los posibles destinos en el map.*
- **self.new_car_spawns**: *Es una lista con las posibles posiciones en las que un carro nuevo puede aparecer.*

Las funciones de cada método son las siguientes:

- **get_unique_pos:** *Devuelve una posición donde no se encuentre algún carro, edificio, destino o semáforo.*
- **get_unique_destination:** *Devuelve una posición de destino que no se encuentre actualmente utilizada por un carro.*

Finalmente, en la inicialización de la clase **TrafficModel** se parametriza el atributo ***MAX_STEPS***, que define el número máximo de pasos de tiempo que el modelo puede llevar a cabo. Este es el único parámetro necesario para crear una instancia del modelo. Al momento de inicializarse, el modelo lee un mapa en formato .txt que contiene las direcciones que los carros deben de seguir, así como las posiciones de los edificios y los semáforos, agregando una instancia de cada agente correspondiente a la instancia de ***MultiGrid*** contenida en el modelo. Así mismo, usando el mismo archivo de texto, se crea un grafo dirigido que conecta únicamente a los nodos que sigan las direcciones señaladas por el mapa, el cual se implementó utilizando las siguientes librerías:

In [21]:
from enum import Enum, auto
from mesa import Model
from directions import Directions as dirs

Para el grafo que representa al mapa, se crearon las clases ***Node***, que representa a los nodos y ***NodeTypes***, que representa a los tipos de Nodo (*esto es, el estado de cada nodo para el algoritmo A-Star*).

In [22]:
class NodeTypes(Enum):
    START = auto()
    END = auto()
    OBSTACLE = auto()
    CLOSED = auto()
    UNVISITED = auto()
    VISITED = auto()


class Node:
    def __init__(self, col: int, row: int, model: Model) -> None:
        self.col = col
        self.row = row
        self.state = NodeTypes.UNVISITED
        self.direction = None
        self.model = model
        self.neighbors = []
    
    # Update the adjacent nodes of a node
    def update_adj(self) -> None:
        # Update every adjacent node
        self.model.standard_map[self.row][self.col + 1].update_neighbors()
        self.model.standard_map[self.row][self.col - 1].update_neighbors()
        self.model.standard_map[self.row + 1][self.col].update_neighbors()
        self.model.standard_map[self.row - 1][self.col].update_neighbors()
    
    # Update the neighbor list of a node
    def update_neighbors(self) -> None:
        self.neighbors.clear()
        if self.col < self.model.cols - 1 and not self.model.standard_map[self.row][self.col + 1].state == NodeTypes.OBSTACLE:
            if self.model.standard_map[self.row][self.col].direction != dirs.LEFT:
                if not (self.model.standard_map[self.row][self.col].direction == dirs.UP and self.model.standard_map[self.row][self.col + 1].direction == dirs.LEFT):
                    self.neighbors.append(self.model.standard_map[self.row][self.col + 1])
        if self.col > 0 and not self.model.standard_map[self.row][self.col - 1].state == NodeTypes.OBSTACLE:
            if self.model.standard_map[self.row][self.col].direction != dirs.RIGHT:
                if not (self.model.standard_map[self.row][self.col].direction == dirs.DOWN and self.model.standard_map[self.row][self.col - 1].direction == dirs.RIGHT):
                    self.neighbors.append(self.model.standard_map[self.row][self.col - 1])
        if self.row < self.model.rows - 1 and not self.model.standard_map[self.row + 1][self.col].state == NodeTypes.OBSTACLE:
            if self.model.standard_map[self.row][self.col].direction != dirs.UP:
                if not (self.model.standard_map[self.row][self.col].direction == dirs.RIGHT and self.model.standard_map[self.row + 1][self.col].direction == dirs.UP) and not (self.model.standard_map[self.row][self.col].direction == dirs.LEFT and self.model.standard_map[self.row + 1][self.col].direction == dirs.UP):
                    self.neighbors.append(self.model.standard_map[self.row + 1][self.col])
        if self.row > 0 and not self.model.standard_map[self.row - 1][self.col].state == NodeTypes.OBSTACLE:
            if self.model.standard_map[self.row][self.col].direction != dirs.DOWN:
                if not (self.model.standard_map[self.row][self.col].direction == dirs.RIGHT and self.model.standard_map[self.row - 1][self.col].direction == dirs.DOWN) and not (self.model.standard_map[self.row][self.col].direction == dirs.LEFT and self.model.standard_map[self.row - 1][self.col].direction == dirs.DOWN):
                    self.neighbors.append(self.model.standard_map[self.row - 1][self.col])

El nodo tienes los siguientes métodos:

- **update_adj:** *Actualiza los vecinos únicamente de los nodos adyacentes.*
- **update_neighbors:** *Actualiza la lista de vecinos de un nodo, conectando únicamente a aquellos que representan una dirección válida a la que un carro puede moverse dependiendo de la dirección en la que actualmente se encuentre.*

Finalmente, en el mismo módulo ***grid_manager***, se encuentran los siguientes métodos:

In [23]:
# Update every node's neighbor list
def init_neighborhood(grid: list) -> None:
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            grid[i][j].update_neighbors()
            

# Create a grid with empty nodes
def make_grid(rows: int, cols: int, model: Model) -> list:
    grid = []
    for row in range(rows):
        temp = []
        for col in range(cols):
            node = Node(col, row, model)
            temp.append(node)
        grid.append(temp)
    return grid


# Get Manhattan distance from one node to another
def h(n1: Node, n2: Node):
    # Return the sum of the absolute value of the substraction of the x and y distances between two nodes
    return abs(n1.col - n2.col) + abs(n1.row - n2.row)

Estos métodos tienen la función de inicializar el grafo (*creando nodos vacíos dentro del grafo*), actualizar los vecinos de cada nodo dentro del grafo, y encontrar las distancias Manhattan entre dos nodos (*es decir, suma de la distancia horizontal y vertical entre ellos*), respectivamente.

### **Etapa 1.2:** *Modelación de los agentes*

Para la modelación del agente, no se usaron más librerías que las anteriormente mencionadas, sin embargo, se importó un módulo propio para que cada agente genere el camino más corto a su destino utilizando mediante el método A*.

In [24]:
from astar import *

Este algoritmo funciona de tal manera que, dado un grafo con dirección, se exploren sus nodos desde un nodo inicial hacia un nodo final conocido, moviéndose utilizando una función que calcule la suma de la distancia entre ambos nodos más la distancia Manhattan entre el nodo actualmente evaluado y el nodo final. Estas evaluaciones se realizan utilizando una estructura llamada ***Priority Queue***, que representa a los nodos que se están visitando. A fin de encontrar el camino más corto, cada nodo elegirá moverse hacia el nodo siguiente con menor valor heurístico (*nuevamente, la suma de la distancia entre ambos nodos y la distancia entre el siguiente nodo y el nodo de destino*). El algoritmo se implementó de la siguiente manera:

In [25]:
from queue import PriorityQueue

# Get the position of each node in a dictionary containing the nodes that were visited as value and the node from which they came from as 
# the key.
def get_nodes_in_path(came_from: dict, current: Node):
    # Create an empty list to store the positions of each node
    path = []
    # While there's still a next pair on the dictionary:
    while current in came_from:
        # Add the position as a tuple to the path list
        path.append((current.row, current.col))
        # Change current node
        current = came_from[current]
    # Put the list in the right order to be followed
    path.reverse()
    return path        


def get_shortest_path(grid: list, start: Node, end: Node, model: Model) -> list or None:
    # Open destination Node momentarily so the algorithm can detect it as a waypoinr
    model.standard_map[end.row][end.col].state = NodeTypes.END
    model.standard_map[end.row][end.col].update_adj()
    # Initialize counter, queue and set
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    # Initialize each cell's f score and g score
    g_score = {node: float("inf") for row in grid for node in row}
    f_score = {node: float("inf") for row in grid for node in row}
    # Declare start node's scores
    g_score[start] = 0
    f_score[start] = h(start, end)
    # Returns nodes in priority queue
    open_set_hash = {start}
    # Run until the open set is empty
    while not open_set.empty():
        # Current node will be the start node
        current = open_set.get()[2]
        # Remove current node from the open set hash
        open_set_hash.remove(current)
        # Check if current node is already the destination
        if current == end:
            model.standard_map[end.row][end.col].state = NodeTypes.OBSTACLE
            model.standard_map[end.row][end.col].update_adj()
            return get_nodes_in_path(came_from, current)
        # Check the neighbors of the current node and add a temporary g score
        for neighbor in current.neighbors:
            # Check each neighbor's g score and look for the smallest one
            temp_g = g_score[current] + 1
            if temp_g < g_score[neighbor]:
                # Tell program that the current path comes from the current node
                came_from[neighbor] = current
                # Set the neighbor's g score the new g score
                g_score[neighbor] = temp_g
                f_score[neighbor] = temp_g + h(neighbor, end)
                # If neighbor has not been visited, change it's state and add it to the priority queue
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.state = NodeTypes.VISITED
                if current != start:
                    current.state = NodeTypes.CLOSED
    # Close the destination state by setting it as an obstacle
    model.standard_map[end.row][end.col].state = NodeTypes.OBSTACLE
    # Rearrange the adjacent nodes
    model.standard_map[end.row][end.col].update_adj()
    return []

Los agentes se implementaron de la siguiente manera:

El agente ***Building*** fue creado para tener la capacidad de representarlo usando los módulos de visualización de mesa, su atributo **type_id** permite identificar que se trata de este tipo de agente con mayor facilidad.

In [26]:
class Building(Agent):
    def __init__(self, unique_id: int, model: Model) -> None:
        super().__init__(unique_id, model)
        self.type_id = agt.BUILDING

La clase ***Light*** 

In [27]:
class Light(Agent):
    def __init__(self, unique_id: int, model: Model, direction: Directions) -> None:
        super().__init__(unique_id, model)
        self.type_id = agt.LIGHT
        self.direction = direction
        self.state = False if direction == dirs.UP or direction == dirs.DOWN else True

In [28]:
class Destination(Agent):
    def __init__(self, unique_id: int, model: Model) -> None:
        super().__init__(unique_id, model)
        self.type_id = agt.DESTINATION
        self.occupied = False

In [29]:
class Building(Agent):
    def __init__(self, unique_id: int, model: Model) -> None:
        super().__init__(unique_id, model)
        self.type_id = agt.BUILDING


class Light(Agent):
    def __init__(self, unique_id: int, model: Model, direction: Directions) -> None:
        super().__init__(unique_id, model)
        self.type_id = agt.LIGHT
        self.direction = direction
        self.state = False if direction == dirs.UP or direction == dirs.DOWN else True
        self.cars_in_col = 0
        self.cars_in_row = 0


class Destination(Agent):
    def __init__(self, unique_id: int, model: Model) -> None:
        super().__init__(unique_id, model)
        self.type_id = agt.DESTINATION
        self.occupied = False


class Car(Agent):
    def __init__(self, unique_id: int, model: Model, start_pos: tuple) -> None:
        super().__init__(unique_id, model)
        # Set initial attributes
        self.type_id = agt.CAR
        self.destination = self.random.choice(self.model.destinations)
        self.has_arrived = False
        # Set start and end in map
        self.map = model.standard_map
        self.map[start_pos[0]][start_pos[1]].state = NodeTypes.START
        self.map[self.destination.pos[0]][self.destination.pos[1]].state = NodeTypes.END
        # Find path to destination
        self.path = get_shortest_path(self.map, self.map[start_pos[0]][start_pos[1]], self.map[self.destination.pos[0]][self.destination.pos[1]], model)
        self.next_cell = None
        self.last_dir = self.map[start_pos[0]][start_pos[1]].direction
        self.turn_dir = None
        self.main_av = False

    # Check if destination cell is within the neighborhood
    def check_destination(self, neighborhood: list) -> tuple:
        # Check every cell in neighborhood
        for cell in neighborhood:
            # Check the content in every cell
            content = self.model.grid.get_cell_list_contents(cell)
            # Check every agent within the cell's content
            for agent in content:
                # Set next position as the destination cell
                if agent.type_id == agt.DESTINATION:
                    return agent.pos    
        # If nothing was found, don't return anything
        return None
    
    # Return whether the car is on a main avenue or not
    def is_in_main_av(self):
        # If car is in the first two columns or in the last two columns, return True
        if (self.pos[0] >= 0 and self.pos[0] < 2) and (self.pos[0] >= self.model.cols - 2 and self.pos[0] < self.model.cols):
            return True
        # If car is in the first two rows or in the last two rows, return True
        elif (self.pos[1] >= 0 and self.pos[1] < 2) and (self.pos[1] >= self.model.rows - 2 and self.pos[1] < self.model.rows):
            return True
        # Otherwise, return False
        else:
            return False
    
    # Get the direction of the car's next move
    def get_turn_dir(self):
        return (self.next_cell[0] - self.pos[0], self.next_cell[1] - self.pos[1])
    
    # Check if a car will let other car go first in a cell
    def give_priority(self, other: Agent):
        # If the other car is going straight and this car is going to turn
        if other.turn_dir == other.last_dir and self.turn_dir != self.last_dir:
            return True
        # If both cars are going to turn
        elif other.turn_dir == other.last_dir and self.turn_dir != self.last_dir:
            return False
        # If both cars are turning or going straight
        else:
            # If this car comes from the main avenue, don't let the other pass and viceversa
            return not self.main_av

    # Check if next cell is being targeted by other agents and return if car can go to it
    def can_get_to_next_cell(self) -> bool:
        # If next cell is not the car's destination, check if it can move towards it
        if self.next_cell != self.destination.pos:
            # If next cell is empty:
            if self.model.grid.is_cell_empty(self.next_cell):
                # Get next cell's neighbors
                next_neighbors = self.model.grid.get_neighborhood(self.next_cell, moore=False, include_center=False)
                # Remove self position from the neighborhood
                if self.pos in next_neighbors:
                    next_neighbors.remove(self.pos)
                # Check each cell within the neighborhood
                for cell in next_neighbors:
                    # Check each cell's contents
                    for agent in self.model.grid.get_cell_list_contents(cell):
                        # If there's a car in that cell, check if it's going to this cell too
                        if agent.type_id == agt.CAR:
                            if not agent.has_arrived:
                                # If they're going to the same cell, check if this car will let the other go first
                                if agent.next_cell == self.next_cell:
                                    return not self.give_priority(agent)
                # If no agents were found in that cell, let the car advance
                return True
            # Otherwise, check if the contents of the next cell contain a car
            else:
                contents = self.model.grid.get_cell_list_contents(self.next_cell)
                for agent in contents:
                    # If agent is a car, don't let it advance
                    if agent.type_id == agt.CAR:
                        return False
                # If no car was found, let the car advance
                return True
        # Else, just return True since it can move no matter what
        else:
            return True
            
    # Check if there are cars or lights in front of the car
    def has_green_light(self) -> bool:
        # If there's a light in the next cell, check if it's a light
        content = self.model.grid.get_cell_list_contents(self.next_cell)
        for agent in content:
            # If agent is a light, return it's state
            if agent.type_id == agt.LIGHT:
                return agent.state
        # If there is no light, advance to next cell
        return True

    # Try to move to the next cell
    def move_next(self) -> None:
        # Check if any other car is directed to the next cell
        if self.can_get_to_next_cell():
            # If there's a green light, advance to it
            if self.has_green_light():
                # Remove waypoint from path
                self.path.pop(0)
                # Move agent towards the next cell
                self.model.grid.move_agent(self, self.next_cell)
    
    # Return a boolean value representing if two cars are in the same position 
    def check_crashes(self):
        # Get contents from current cell
        cell_content = self.model.grid.get_cell_list_contents(self.pos)
        # Remove self from content
        cell_content.remove(self)
        # Check if there's content in the cell other than self
        if len(cell_content) > 0:
            # Check every agent that is a car
            for agent in cell_content:
                if agent.type_id == agt.CAR:
                    # If the car's position is already in the car's definition, return false
                    if agent.pos == agent.destination.pos:
                        return False
                    # Otherwise, return true
                    else:
                        print(f"Crash in {self.pos}")
                        return True
            return False
        # Otherwise return false
        else:
            return False

    def step(self) -> None:
        # If car hasn't arrived to it's destination
        if not self.has_arrived:
            # Set car's current direction
            self.last_dir = self.map[self.pos[0]][self.pos[1]].direction
            # If path still has remaining cells, assign the first one to the car's next cell
            if len(self.path) > 0:
                # Remove the next cell from the path
                self.next_cell = self.path[0]
                self.turn_dir = self.get_turn_dir()
                self.main_av = self.is_in_main_av()
                self.turn_dir = self.map[self.next_cell[0]][self.next_cell[1]].direction
                # Try to move to the next cell
                self.move_next()
                # Check if car is in the same position as another car
                self.model.running = not self.check_crashes()
            # Otherwise, set the car's has_arrived attribute to True
            else:
                self.model.arrivals += 1
                self.has_arrived = True